# Estimating the total biomass of terrestrial protists
After searching the literature, we could not find a comprehensive account of the biomass of protists in soils. We generated a crude estimate of the total biomass of protists in soil based on two main methodologies. The first is based on direct counts of protists in soils, and the second is based on molecular techniques. We detail below the calculation of the global protist biomass using each method. Our best estimate for the total biomass of soil protists is the geometric mean of the estimates from the five different methodologies.

## Direct counts of protists
In this method, in order to calculate the total biomass of soil protists we calculate a characteristic number of individual protists for each one of the morphological groups of protists (flagellates, ciliates, and naked and testate ameobae). We combine these estimates with estimates for the carbon content of each morphological group.

### Number of protists
To estimate the total number of protists, we assembled data on the number of protists in soils which contains 128 measuements from 22 independent studies. Here is a sample of the data:

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import gmean
import sys
sys.path.insert(0,'../../statistics_helper/')
from fraction_helper import *
from CI_helper import *
pd.options.display.float_format = '{:,.1e}'.format
data = pd.read_excel('terrestrial_protist_data.xlsx')
data.head()

,Reference,DOI,Habitat,Site,Number of naked amoebae [# g^-1],Number of ciliates [# g^-1],Number of testate amoebae [# g^-1],Number of flagellates [# g^-1],Remarks
0,Robinson et al.,http://dx.doi.org/10.1111/j.1550-7408.2002.tb0...,Desert,Australia,6.1e+03,1.5e+02,4.9e+03,nan,"Samples from termite mound dropped, taken from..."
1,Robinson et al.,http://dx.doi.org/10.1111/j.1550-7408.2002.tb0...,Desert,Australia,1.3e+04,1.5e+02,2.7e+03,nan,"Samples from termite mound dropped, taken from..."
2,Robinson et al.,http://dx.doi.org/10.1111/j.1550-7408.2002.tb0...,Desert,Australia,4.3e+03,6.0e+01,2.3e+03,nan,"Samples from termite mound dropped, taken from..."
3,Robinson et al.,http://dx.doi.org/10.1111/j.1550-7408.2002.tb0...,Desert,Australia,3.0e+04,2.7e+02,2.2e+03,nan,"Samples from termite mound dropped, taken from..."
4,Robinson et al.,http://dx.doi.org/10.1111/j.1550-7408.2002.tb0...,Desert,Australia,4.0e+03,3.8e+02,4.0e+03,nan,"Samples from termite mound dropped, taken from..."


To estimate the total number of protists, we group our samples to different habitats and to the study in which they were taken. We calculate the characteristic number of each of the groups of protists per gram of soil. To do this we first derive a representative value for each study in case there was more than one measurement done in it. We calculate the representative value for each study in each habitat. Then we calculate the average of different representative values from different studies within the same habitat. We calculate the averages either by using the arithmetic mean or the geometric mean.

In [2]:
# Define the function ot calculate the geometric mean of number of each group of protists per gram
def groupby_gmean(input):
    return pd.DataFrame({'Number of ciliates [# g^-1]': gmean(input['Number of ciliates [# g^-1]'].dropna()),
                        'Number of naked amoebae [# g^-1]': gmean(input['Number of naked amoebae [# g^-1]'].dropna()),
                        'Number of testate amoebae [# g^-1]': gmean(input['Number of testate amoebae [# g^-1]'].dropna()),
                        'Number of flagellates [# g^-1]': gmean(input['Number of flagellates [# g^-1]'].dropna())},index=[0])

# Define the function ot calculate the arithmetic mean of number of each group of protists per gram
def groupby_mean(input):
    return pd.DataFrame({'Number of ciliates [# g^-1]': np.nanmean(input['Number of ciliates [# g^-1]'].dropna()),
                        'Number of naked amoebae [# g^-1]': np.nanmean(input['Number of naked amoebae [# g^-1]'].dropna()),
                        'Number of testate amoebae [# g^-1]': np.nanmean(input['Number of testate amoebae [# g^-1]'].dropna()),
                        'Number of flagellates [# g^-1]': np.nanmean(input['Number of flagellates [# g^-1]'].dropna())},index=[0])

# Group the samples by habitat and study, and calculate the geometric mean
grouped_data_gmean = data.groupby(['Habitat','DOI']).apply(groupby_gmean)

# Group the samples by habitat and study, and calculate the arithmetic mean
grouped_data_mean = data.groupby(['Habitat','DOI']).apply(groupby_mean)

# Group the representative values by habitat, and calculate the geometric mean
habitat_gmean = grouped_data_gmean.groupby('Habitat').apply(groupby_gmean)

# Group the representative values by habitat, and calculate the arithmetic mean
habitat_mean = grouped_data_mean.groupby('Habitat').apply(groupby_mean)

habitat_gmean.set_index(habitat_gmean.index.droplevel(1),inplace=True)
habitat_mean.set_index(habitat_mean.index.droplevel(1),inplace=True)

/usr/local/lib/python3.5/dist-packages/scipy/stats/stats.py:315: RuntimeWarning: Mean of empty slice.
  return np.exp(log_a.mean(axis=axis))
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: Mean of empty slice
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: Mean of empty slice
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: Mean of empty slice
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: Mean of empty slice
  if sys.path[0] == '':


Here is the calculated geometric mean number of cells per gram for each habitat and each group of protists:

In [3]:
habitat_gmean

,Number of ciliates [# g^-1],Number of flagellates [# g^-1],Number of naked amoebae [# g^-1],Number of testate amoebae [# g^-1]
Habitat,,,,
Cropland,3.0e+02,3.6e+03,6.2e+03,2.7e+03
Desert,1.6e+02,nan,8.9e+03,3.3e+03
General,nan,1.0e+06,1.0e+05,nan
Grassland,2.9e+02,nan,2.1e+04,3.5e+03
Shrubland,7.2e+01,nan,nan,9.6e+03
Temperate Forest,4.0e+02,1.2e+06,8.2e+04,4.1e+03
Tropical Forest,nan,nan,nan,4.2e+04
Tundra,nan,1.1e+06,nan,1.5e+03
Woodland,2.6e+02,nan,nan,1.8e+04


For some groups, not all habitats have values. We fill values for missing data by the following scheme. If we have data for the group of protists from the "General" habitat, which is based on expert assesment of the characteristic number of individuals for that group per gram of soil, we fill the missing values with the value for the "General" habitat.

The only other missing data was for ciliates in tropical forests and tundra. For tropical forest, we used the values from temperate forests forests. For tundra, we use the mean over all the different habitats to fill the value:

In [4]:
# Fill missing values for naked amoebae
habitat_mean.loc[['Shrubland','Tropical Forest','Tundra','Woodland'],'Number of naked amoebae [# g^-1]'] = habitat_mean.loc['General','Number of naked amoebae [# g^-1]']
habitat_gmean.loc[['Shrubland','Tropical Forest','Tundra','Woodland'],'Number of naked amoebae [# g^-1]'] = habitat_gmean.loc['General','Number of naked amoebae [# g^-1]']

# Fill missing values for flagellates
habitat_gmean.loc[['Desert','Grassland','Shrubland','Tropical Forest','Woodland'],'Number of flagellates [# g^-1]'] = habitat_gmean.loc['General','Number of flagellates [# g^-1]']
habitat_mean.loc[['Desert','Grassland','Shrubland','Tropical Forest','Woodland'],'Number of flagellates [# g^-1]'] = habitat_mean.loc['General','Number of flagellates [# g^-1]']

# Fill missing values for ciliates
habitat_gmean.loc['Tropical Forest','Number of ciliates [# g^-1]'] = habitat_gmean.loc['Temperate Forest','Number of ciliates [# g^-1]']
habitat_mean.loc['Tropical Forest','Number of ciliates [# g^-1]'] = habitat_mean.loc['Temperate Forest','Number of ciliates [# g^-1]']
habitat_gmean.loc['Tundra','Number of ciliates [# g^-1]'] = gmean(habitat_mean['Number of ciliates [# g^-1]'].dropna())
habitat_mean.loc['Tundra','Number of ciliates [# g^-1]'] = habitat_mean['Number of ciliates [# g^-1]'].dropna().mean()

We convert the total number of individuals per gram of soil in each habitat to the total number of individuals per unit area. To do this we use a soil bulk density of ≈1.5 g cm$^{-3}$, and assume that most biomass is concentrated in the top 20 cm of soil (see the section on terrestrial protists for in the Supplemenray Information for details):

In [5]:
bulk_density = 1.5e6
biomass_depth = 0.2
habitat_per_m2_gmean = habitat_gmean*bulk_density*biomass_depth
habitat_per_m2_mean = habitat_mean*bulk_density*biomass_depth

To calculate the total number of protists we multiply the total number of individuals per unit area of each type of protist in each habitat by the total area of each habitat taken from the book [Biogeochemistry: An analysis of Global Change](https://www.sciencedirect.com/science/book/9780123858740) by Schlesinger & Bernhardt. The areas of each habitat are:

In [6]:
habitat_area = pd.read_excel('terrestrial_protist_data.xlsx','Biome area', skiprows=1,index_col=0)
habitat_area

,Area [m^2],Unnamed: 2
Biome,,
Boreal Forest,13700000000000,"Temperate forest, Tropical rainforest"
Desert,27700000000000,Desert
Temperate Forest,10400000000000,Temperate forest
Grassland,15000000000000,Grassland
Tropical Forest,17500000000000,Tropical rainforest
Tundra,5600000000000,Tundra
Tropical Savanna,27700000000000,"Scrubland,grassland,Temprate Forest, Tropical ..."
Cropland,15500000000000,Cropland


There are some habitats for which we do not have data, namely boreal forests and savanna. For the Boreal forest habitat we use the value for the temperate forest habitat. For the savanna habitat we use the mean of the values for the tropical forest, woodland, shrubland and grassland.

In [7]:
habitat_per_m2_gmean.loc['Boreal Forest'] = habitat_per_m2_gmean.loc['Temperate Forest']
habitat_per_m2_mean.loc['Boreal Forest'] = habitat_per_m2_mean.loc['Temperate Forest']

habitat_per_m2_gmean.loc['Tropical Savanna'] = gmean(habitat_per_m2_gmean.loc[['Tropical Forest','Woodland','Shrubland','Grassland']])
habitat_per_m2_mean.loc['Tropical Savanna'] = habitat_per_m2_gmean.loc[['Tropical Forest','Woodland','Shrubland','Grassland']].mean(axis=0)

tot_num_gmean = habitat_per_m2_gmean.mul(habitat_area['Area [m^2]'],axis=0)
tot_num_mean = habitat_per_m2_mean.mul(habitat_area['Area [m^2]'],axis=0)
print(tot_num_mean.sum())
print(tot_num_gmean.sum())
print(gmean([tot_num_mean.sum(),tot_num_gmean.sum()]))

Number of ciliates [# g^-1]          3.5e+22
Number of flagellates [# g^-1]       7.2e+25
Number of naked amoebae [# g^-1]     2.4e+24
Number of testate amoebae [# g^-1]   7.6e+23
dtype: float64
Number of ciliates [# g^-1]          1.2e+22
Number of flagellates [# g^-1]       3.7e+25
Number of naked amoebae [# g^-1]     2.1e+24
Number of testate amoebae [# g^-1]   4.1e+23
dtype: float64
[  2.03075171e+22   5.17113847e+25   2.24009937e+24   5.57197687e+23]


In [8]:
tot_num_protist = gmean([tot_num_mean.sum(),tot_num_gmean.sum()])
tot_num_protist

array([  2.03075171e+22,   5.17113847e+25,   2.24009937e+24,
         5.57197687e+23])

### Carbon content of protists
We estimate the characteristic carbon content of a single protist from each of the morphological groups of protists  based on data from several sources. Here is a sample of the data:

In [18]:
cc_data =  pd.read_excel('terrestrial_protist_data.xlsx', 'Carbon content')
cc_data.head()

,Reference,DOI,Carbon content of naked amoebae [g C cell^-1],Carbon content of ciliates [g C cell^-1],Carbon content of testate amoebae [g C cell^-1],Carbon content of flagellates [g C cell^-1],Remarks
0,Wanner et al.,http://dx.doi.org/10.1007/s00248-007-9322-2,nan,nan,1.5e-09,nan,"Calculated from table 2, assuming 15% carbon c..."
1,Wanner et al.,http://dx.doi.org/10.1007/s00248-007-9322-2,nan,nan,1.1e-09,nan,"Calculated from table 2, assuming 15% carbon c..."
2,Wanner et al.,http://dx.doi.org/10.1007/s00248-007-9322-2,nan,nan,2.1e-09,nan,"Calculated from table 2, assuming 15% carbon c..."
3,Wanner et al.,http://dx.doi.org/10.1007/s00248-007-9322-2,nan,nan,3.4e-09,nan,"Calculated from table 2, assuming 15% carbon c..."
4,Foissner,http://dx.doi.org/10.1016/0167-8809(92)90093-Q,nan,3.8e-09,4.7e-09,nan,"Calculated from table 2, assuming 15% carbon c..."


We combine this data with an additional source from [Finlay & Fenchel](http://dx.doi.org/10.1078/1434-4610-00060). We calculate the average cell length for each group. 

In [10]:
# Load data from Finlay & Fenchel
ff_data = pd.read_excel('terrestrial_protist_data.xlsx', 'Finlay & Fenchel', skiprows=1)

# Define the function to calculate the weighted average for each group of protists
def weighted_av_groupby(input):
    return np.average(input['Length [µm]'],weights=input['Abundance [# g^-1]'])

cell_lengths = ff_data.groupby('Protist type').apply(weighted_av_groupby)

We convert the cell length to biovolume according the the allometric relation decribed in Figure 10 of Finlay & Fenchel. The relation between cell volume and cell length is given by the equation: 
$$V = 0.6×L^{2.36}$$
Where V is the cell volume in $µm^3$ and L is the cell length in µm.

In [11]:
cell_volumes = 0.6*cell_lengths**2.36
cell_volumes

Protist type
Ciliate           5.4e+03
Flagellate        1.2e+02
Naked amoebae     1.4e+03
Testate amoebae   3.6e+03
dtype: float64

We convert cell volumes to carbon content assuming ≈150 fg C µm$^3$:

In [12]:
ff_carbon_content = cell_volumes*150e-15
pd.options.display.float_format = '{:,.1e}'.format
ff_carbon_content

Protist type
Ciliate           8.1e-10
Flagellate        1.8e-11
Naked amoebae     2.0e-10
Testate amoebae   5.5e-10
dtype: float64

We add these numbers as an additional source for calculating the carbon content of protists:

In [13]:
cc_data.loc[cc_data.index[-1]+1] = pd.Series({'Reference': 'Finlay & Fenchel',
                   'DOI': 'http://dx.doi.org/10.1078/1434-4610-00060',
                   'Carbon content of ciliates [g C cell^-1]': ff_carbon_content.loc['Ciliate'],
                   'Carbon content of naked amoebae [g C cell^-1]': ff_carbon_content.loc['Naked amoebae'],
                   'Carbon content of testate amoebae [g C cell^-1]': ff_carbon_content.loc['Testate amoebae'],
                   'Carbon content of flagellates [g C cell^-1]': ff_carbon_content.loc['Flagellate']
                  })


We calculate the geometric mean of carbon contents for first for values within each study and then for the average values between studies:

In [14]:
def groupby_gmean(input):
    return pd.DataFrame({'Carbon content of ciliates [g C cell^-1]': gmean(input['Carbon content of ciliates [g C cell^-1]'].dropna()),
                        'Carbon content of naked amoebae [g C cell^-1]': gmean(input['Carbon content of naked amoebae [g C cell^-1]'].dropna()),
                        'Carbon content of testate amoebae [g C cell^-1]': gmean(input['Carbon content of testate amoebae [g C cell^-1]'].dropna()),
                        'Carbon content of flagellates [g C cell^-1]': gmean(input['Carbon content of flagellates [g C cell^-1]'].dropna())},index=[0])


study_mean_cc = cc_data.groupby('DOI').apply(groupby_gmean)
mean_cc = study_mean_cc.reset_index().groupby('level_1').apply(groupby_gmean)

/usr/local/lib/python3.5/dist-packages/scipy/stats/stats.py:315: RuntimeWarning: Mean of empty slice.
  return np.exp(log_a.mean(axis=axis))
/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [15]:
gmean(study_mean_cc['Carbon content of flagellates [g C cell^-1]'].dropna())
mean_cc

,,Carbon content of ciliates [g C cell^-1],Carbon content of flagellates [g C cell^-1],Carbon content of naked amoebae [g C cell^-1],Carbon content of testate amoebae [g C cell^-1]
level_1,,,,,
0,0,7.0e-10,2.2e-11,2.0e-10,1.3e-09


In [16]:
(tot_num_protist*mean_cc).sum(axis=1)

level_1   
0        0   2.4e+15
dtype: float64

In [17]:


# Define the function to calculate the geometric mean for each study
def groupby_gmean(input):
    mean = gmean(input['Biomass density [g C m^-2]'])
    habitat = np.unique(input['Habitat'])[0]
    return pd.Series({'Habitat': habitat, 'Biomass density [g C m^-2]': mean})

# Calculate the geometric mean for each study
study_mean = data.groupby('Reference').apply(groupby_gmean)

# Calculate the geometric mean of the biomass density at each habitat
habitat_mean = data.groupby('Habitat')['Biomass density [g C m^-2]'].apply(gmean)

# Calculate the geometric mean of biomass densities from different habitats
direct_biomass_mean = gmean(habitat_mean)

print('Our best estimate for the biomass density of  protists in soil based on direct biomass density measurements is ≈%.1f g C m^-2' % direct_biomass_mean)

KeyError: 'Biomass density [g C m^-2]'

To generate our estimate for the total biomass of protists using the direct biomass density measurement method, we multiply the our best estimate for the biomass density by the total area of ice-free land surface, which is ≈$1.3×10^{14} m^2$:

In [ ]:
ice_free_area = 1.3e14

# Calculate the total biomass of soil protists
method1_estimate = direct_biomass_mean*ice_free_area 

print('Our best estimate for the biomass of soil protists using direct biomass density measurements is ≈%.1f Gt C' % (method1_estimate/1e15))

## Number of individuals and carbon content
In this method, in order to calculate the total biomass of soil protists we calculate a characteristic number of individual protists in a gram of soil for each one of the morphological groups of protists (flagellates, ciliates, and naked and testate ameobae). We combine these estimates with estimates for the carbon content of each morphological group.

### Characteristic carbon content of protists
We estimate the characteristic carbon content of a single protist from each of the morphological groups of protists  based on data from three sources.

The first source is [Finlay & Fenchel](http://dx.doi.org/10.1078/1434-4610-00060). We calculate the average cell length for each group. 

For flagellates, the estimates on the number of individuals per gram of soil distinguishes between small and large flagellates (defined as flagellates below or above 15 µm in diameter). We thus estimate the average length of small and large flagellates by dividing them into these two size categories.

In [ ]:
# Load data from Finlay & Fenchel
ff_data = pd.read_excel('terrestrial_protist_data.xlsx', 'Finlay & Fenchel', skiprows=1)

# Define the function to calculate the weighted average for each group of protists
def weighted_av_groupby(input):
    return np.average(input['Length [µm]'],weights=input['Abundance [# g^-1]'])

cell_lengths = ff_data.groupby('Protist type').apply(weighted_av_groupby)

We convert the cell length to biovolume according the the allometric relation decribed in Figure 10 of Finlay & Fenchel. The relation between cell volume and cell length is given by the equation: 
$$V = 0.6×L^{2.36}$$
Where V is the cell volume in $µm^3$ and L is the cell length in µm.

In [ ]:
cell_volumes = 0.6*cell_lengths**2.36
cell_volumes

We convert cell volumes to carbon content assuming ≈150 fg C µm$^3$:

In [ ]:
ff_carbon_content = cell_volumes*150e-15
pd.options.display.float_format = '{:,.1e}'.format
ff_carbon_content

Our second source for estimating the carbon content of soil protists is [Persson et al.](http://www.jstor.org/stable/20112829), which reports the dry weight of individuals from different morphological types:

In [ ]:
persson_data = pd.read_excel('terrestrial_protist_data.xlsx', 'Persson', skiprows=1)
persson_data

Our third source is [Schaefer](http://dx.doi.org/10.1007/BF00318544), which reports the total number of cells and the total biomass for three morphological groups of protists - Flagellates, Amoebae and Testate amoebae. We calculate the characteristic carbon content for each group by dividing the total biomass by the total number of individuals:

In [ ]:
# Load the data from Schaefer
schaefer_data = pd.read_excel('terrestrial_protist_data.xlsx', 'Schaefer', skiprows=1,index_col='Group')

# Calculate the characteristic carbon content for each of the groups of protists
schaefer_cc = schaefer_data['Biomass density [g C m^-2]']/schaefer_data['Number of individuals (# m^-2) ']
schaefer_cc

Our best estimate for the carbon content of each morphological group is the geometric mean of the estimates from Finlay & Fenchel, Persson et al. and Schaefer. Persson reports values for Rhizopoda, which include naked amoebae, and does not report values for Testate amoebae. Schaefer reports values for naked and testate amoebae but not for ciliates. For flagellates, we use only the data from Finlay & Fenchel as it is able to distinguish between small and large Flagellates.

In [ ]:
carbon_content = pd.DataFrame()

ciliate_cc = gmean([ff_carbon_content['Ciliate'],persson_data.loc[0]['Mean body dry weight [g]']/2])
small_flagellate_cc = ff_carbon_content['Small Flagellate']
large_flagellate_cc = ff_carbon_content['Large Flagellate']
naked_amoebae_cc = gmean([ff_carbon_content['Naked amoebae'],persson_data.loc[2]['Mean body dry weight [g]']/2,schaefer_cc.loc['Naked amoebae']])
testate_amoebae_cc = gmean([ff_carbon_content['Testate amoebae'],schaefer_cc.loc['Testate amoebae']])

carbon_content['Carbon content [g C]'] = pd.Series([ciliate_cc,large_flagellate_cc,naked_amoebae_cc,small_flagellate_cc,testate_amoebae_cc])
carbon_content.set_index(ff_carbon_content.index[1:],inplace=True)
carbon_content

### Number of individuals
We rely on two main sources for our estimate. The first is [Adl & Coleman](http://dx.doi.org/10.1007/s00374-005-0009-x). The second source in [Finlay & Fenchel](http://dx.doi.org/10.1078/1434-4610-00060). For each study, we calculate the geometric mean of measurements for each protist group:

In [ ]:
ac_data = pd.read_excel('terrestrial_protist_data.xlsx', 'Adl & Coleman', skiprows=1)

def groupby_mean(input):
    return gmean(input.dropna())
ac_mean = ac_data[['Small flagellates','Large flagellates','Gymnamoebae', 'Ciliates']].apply(groupby_mean)

ff_mean = ff_data.groupby('Protist type')['Abundance [# g^-1]'].apply(sum)

As our best estimate we use the geometric mean of values from Adl & Coleman and from Finlay & Fenchel. For Flagellates, we use only the values from Adl & Coleman, as Finlay & Fenchel rely on the Most Probable Number method to measure the amount of flagellates. This method is based on culturing of protists, which is known to under estimate the actual number of protists. Adl & Coleman do not report a value for testate amoebae, and thus we rely on the values from Finlay & Fenchel.

In [ ]:
abund_mean = pd.DataFrame()
ciliate_abun = gmean([ac_mean['Ciliates'], ff_mean['Ciliate']])
naked_amoebae_abun = gmean([ac_mean['Gymnamoebae'],ff_mean['Naked amoebae']])
abund_mean['Abundance [# g^-1]'] = pd.Series([ciliate_abun,ac_mean['Large flagellates'],naked_amoebae_abun,ac_mean['Small flagellates'], ff_mean['Testate amoebae']])
abund_mean.set_index(ff_mean.index[1:],inplace=True)
abund_mean

To calculate the total biomass of protists per gram of soil, we multiply the total number of individuals of each group of protists by their respective carbon conent, and sum over all protist groups:

In [ ]:
tot_biomass_density = (carbon_content['Carbon content [g C]']*abund_mean['Abundance [# g^-1]']).sum()
print('Our best estimate for the biomass of protists per gram of soil is ≈%.1e g C' % tot_biomass_density)

To convert the biomass density per gram of soil to units of biomass per area, we use a soil bulk density of ≈1.5 g cm$^{-3}$. We assume that most biomass is concentrated in the top 20 cm of soil (see the section on terrestrial protists for in the Supplemenray Information for details).

In [ ]:
bulk_density = 1.5e6
biomass_depth = 0.2
biomass_per_m2 = tot_biomass_density*bulk_density*biomass_depth
print('Our best estimate for the biomass of protists per m^2 of soil is ≈%.0f g C' % biomass_per_m2)
carbon_content['Carbon content [g C]']*abund_mean['Abundance [# g^-1]']*bulk_density*biomass_depth/biomass_per_m2

To calculate the total biomass of protists based on measurements of number of individuals and characteristic carbon contents per individual, we multiply the biomass density per unit area by the total ice-free land surface, which is ≈$1.3×10^{14} m^2$:

In [ ]:
method2_estimate = biomass_per_m2*ice_free_area
print('Our best estimate for the biomass of soil protists using measurements of number of individuals and carbon content is ≈%.1f Gt C' % (method2_estimate/1e15))

The next three methods for estimating the total biomass of protists are based on mulecular surveys of the abundance of protists in soils. The methods we use to estimate the total biomass of protists are 18S rDNA sequencing, 18S rRNA sequencing and metatranscriptomics. 

The molecular techniques we rely on measure the relative fraction of protists out of the total population of soil eukaryotes. Estimating the total biomass of eukaryotes based on molecular techniques assumes a correlation between the number of reads of a specific taxon and its biomass. Even though this procedure is not well established , we rely on it as one of our sources due to the scarcity of data. 

To generate our estimate of the total biomass of soil protist using these molecular techniques, we multiply the fraction of protists out of the total biomass of soil eukaryotes by our estimate for the total biomass of soil fungi, which we assume dominate the biomass of soil eukaryotes.

## 18S rDNA sequencing
To estimate the total biomass of soil protists from 18S rDNA sequencing data, we calculate the fraction of protists out of the total population of soil eukaryotes based on data from forests ([Tedersoo et al.](http://dx.doi.org/10.1038/ismej.2015.116)), grasslands and croplands ([Chen et al.](http://dx.doi.org/10.3389/fmicb.2015.01149)). Below is a sample of the data:


In [ ]:
# Load the data from Chen et al.
chen_data = pd.read_excel('terrestrial_protist_data.xlsx', 'Chen',skiprows=1)
chen_data

We first calculate the geometric mean of the values in Chen et al.:

In [ ]:
chen_mean = frac_mean(chen_data.groupby('Habitat')['Protists'].apply(frac_mean))

As our best estimate for the fraction of protists out of the population of soil eukaryotes we use the geometric mean of the value from Chen et al. and the value reported in Tedersoo et al. of ≈6%. We calculate the total biomass of soil protists by multiplying the fraction of protists out of the total population of soil eukaryotes by the our estimate of the total biomass of soil fungi:

In [ ]:
# The fraction of protists out of the population of soil eukaryotes reported in Tedersoo et al.
tedersoo_frac = 0.06

# Calculate our best estimate for the fraction of soil protists
rDNA_frac = frac_mean(np.array([chen_mean,tedersoo_frac]))

# Our best estimate for the biomass of soil fungi
fungi_biomass = 12e15

# Calculate the total biomass of soil protists based on 18S rDNA sequencing data
method3_estimate = rDNA_frac*fungi_biomass

print('Our best estimate for the biomass of soil protists based on 18S rDNA sequencing data is ≈%.1f Gt C' %(method3_estimate/1e15))

## 18S rRNA sequencing
To estimate the total biomass of soil protists from 18S rRNA sequencing data, we calculate the fraction of protists out of the total population of soil eukaryotes based on data from beech and spruce forests ([Damon et al.](http://dx.doi.org/10.1371/journal.pone.0028967)). Below is a sample of the data:

In [ ]:
# Load the data from Damon et al.
damon_data = pd.read_excel('terrestrial_protist_data.xlsx', 'Damon', skiprows=1)

# Use the data based on 18S rRNA sequencing
rRNA_data = damon_data[damon_data['Method'] == '18S rRNA']
rRNA_data

We calculate the geometric mean of the values from Damon et al. as our best estimate for the fraction of protists out of the total population of soil eukaryotes. We calculate the total biomass of soil protists by multiplying the fraction of protists out of the total population of soil eukaryotes by the our estimate of the total biomass of soil fungi:

In [ ]:
# Calculate the geometric mean of the values from Damon et al.
rRNA_frac = frac_mean(rRNA_data['Fraction of protists'])

# Calculate the total biomass of soil protists based on 18S rRNA sequencing data
method4_estimate = rRNA_frac*fungi_biomass

print('Our best estimate for the biomass of soil protists based on 18S rRNA sequencing data is ≈%.1f Gt C' %(method4_estimate/1e15))

## Metatranscriptomics
To estimate the total biomass of soil protists from metatranscriptomics data, we calculate the fraction of protists out of the total population of soil eukaryotes based on data from beech and spruce forests ([Damon et al.](http://dx.doi.org/10.1371/journal.pone.0028967)). Below is a sample of the data:

In [ ]:
# Use the data based on 18S rRNA sequencing
meta_trans_data = damon_data[damon_data['Method'] == 'Metatranscriptomics']
meta_trans_data

We calculate the geometric mean of the values from Damon et al. as our best estimate for the fraction of protists out of the total population of soil eukaryotes. We calculate the total biomass of soil protists by multiplying the fraction of protists out of the total population of soil eukaryotes by the our estimate of the total biomass of soil fungi:

In [ ]:
# Calculate the geometric mean of the values from Damon et al.
meta_trans_frac = frac_mean(meta_trans_data['Fraction of protists'])

# Calculate the total biomass of soil protists based on 18S rRNA sequencing data
method5_estimate = meta_trans_frac*fungi_biomass

print('Our best estimate for the biomass of soil protists based on metatranscriptomics data is ≈%.1f Gt C' %(method5_estimate/1e15))

As our best estimate for the biomass of soil protists, we use the geometric mean of the five estimates we generated from the five differnt methodologies:

In [ ]:
# Calculate the geometric mean of the five different estimates we generated
best_estimate = gmean([method1_estimate,method2_estimate,method3_estimate,method4_estimate,method5_estimate])

print('Our best estimate for the biomass of terrestrial protists is ≈%.1f Gt C' %(best_estimate/1e15))

# Uncertainty analysis
To assess the uncertainty associated with our estimate of the total biomass of terrestrial protists, we collect available uncertainties for the values reported within studies, between studies using the same method, and between methods. We use the highest uncertainty out of this collection of uncertainties as our best projection for the uncertainty associated wi the estimate of the total biomass of terrestrial protists.

## Intra-study uncertainty
For each study which reports more than one value, we calculate 95% confidence interval of the geometric mean of those values.

### Direct biomass measurement

In [ ]:
# Calculate the 95% confidence interval geometric mean for each study
biomass_study_CI = data.groupby('Reference')['Biomass density [g C m^-2]'].apply(geo_CI_calc)
biomass_study_CI

### Carbon content and number of individuals
We calculate the intra-study 95% confience interval around the estimate of the total number of protists per gram of soil from Adl & Coleman:

In [ ]:
ac_CI = ac_data[['Small flagellates','Large flagellates','Gymnamoebae', 'Ciliates']].apply(geo_CI_calc)
ac_CI

### 18S rDNA sequencing
We calculate the 95% confidence interval for the geometric mean of the values from Chen et al.:

In [ ]:
print('The intra-study uncertainty of the value from Chen et al. is ≈%.1f-fold' %frac_CI(chen_data['Protists']))

### 18S rRNA sequencing

In [ ]:
print('The intra-study uncertainty associated with the fraction of protists based on 18S rRNA sequencing data of Damon et al. is ≈%.2f-fold' %frac_CI(rRNA_data['Fraction of protists']))

### 18S rRNA sequencing

In [ ]:
print('The intra-study uncertainty associated with the fraction of protists based on metatranscriptomics data of Damon et al. is ≈%.1f-fold' %frac_CI(meta_trans_data['Fraction of protists']))

## Intra-methd uncertainty
For each method that relies on more than one study, we calculate the 95% confidence interval of the geometric mean of the values from the different studies. The methods which are based on more than one study are the direct biomass measurement-based method, the carbon content and number of individual based method and the 18S rDNA sequencing-based method.

### Direct biomass measurement
To calculate our best estimate for the biomass of terrestrial protists based on direct biomass density measurements, we first calculated the geometric mean of values from the same habitat, generating characteristic values for each habitat.We then calculate the geomteric mean of the characteristic values for each habitat. 

As a measure of the interstudy uncertainty associated with the estimate based on direct biomass density measurements, we first calculate the 95% confidence interval of the characteristic values for each habitat, and then calculate the 95% confidence invertval around the geometric mean of the characteristic values from each habiat

#### Uncertainty within habitats

In [ ]:
biomass_intra_habitat_CI = data.groupby('Habitat')['Biomass density [g C m^-2]'].apply(geo_CI_calc)
print('The interstudy uncertainty for studies within the same habitat:')
biomass_intra_habitat_CI

#### Uncertainty between habitats

In [ ]:
biomass_inter_habitat_CI = geo_CI_calc(habitat_mean)
print('The 95 percent confidence interval of the geometric mean of the characteristic biomass densities from each habitat is ≈%.1f-fold' %biomass_inter_habitat_CI)

### Carbon content and number of individuals
As a measure of the interstudy uncertainty associated with the estimate of the biomass of terrestrial protists based on the characteristic carbon content of soil protists and the density of number of individuals per unit area, we first calculate the interstudy uncertainty for the characteristic carbon content of each type of protist:

#### Carbon content of groups of protists
For each group of protists, we calculate the 95% confidence interval around our estimate of the characteristic carbon content of single protists from that group. For flagellates, we rely only on a single source, and thus for the estimate of the carbon content of flagellates we are not able to project an uncertainty.

In [ ]:
# Calculate the interstudy 95% confidence interval around the estimate of the carbon content of each group
ciliate_cc_CI = geo_CI_calc([ff_carbon_content['Ciliate'],persson_data.loc[0]['Mean body dry weight [g]']/2])
naked_amoebae_cc_CI = geo_CI_calc([ff_carbon_content['Naked amoebae'],persson_data.loc[2]['Mean body dry weight [g]']/2,schaefer_cc.loc['Naked amoebae']])
testate_amoebae_cc_CI = geo_CI_calc([ff_carbon_content['Testate amoebae'],schaefer_cc.loc['Testate amoebae']])

Next, we calculate the interstudy uncertainty uncertainty associated with the estimate of the total number of individual protists per gram of soil:

#### Number of individuals of 
For each group of protists, we calculate the 95% confidence interval around our estimate of the density of number of individuals from that group per unit area. For flagellates and testate amoebae, we rely only on a single source, and thus we are not able to project an uncertainty.

In [ ]:
# Calculate the interstudy 95% confidence interval around the estimate of the number of individuals
# per gram of soil for each group
ciliate_abun_CI = geo_CI_calc([ac_mean['Ciliates'], ff_mean['Ciliate']])
naked_amoebae_abund_CI = geo_CI_calc([ac_mean['Gymnamoebae'],ff_mean['Naked amoebae']])

We propagate the uncertainties associated with the carbon content and number of individuals per gram soil for each group into our final estimate of the biomass of soil protists. In cases we could not calculate the uncertainty associated with the estimate, we use the mean of the uncertainties from the other groups.

In [ ]:
# Calculate the average uncertainty associated with the estimate of the carbon content 
# and number of individuals per gram of soil
average_cc_CI = np.mean([ciliate_cc_CI,naked_amoebae_cc_CI, testate_amoebae_cc_CI])
average_abund_CI = np.mean([ciliate_abun_CI,naked_amoebae_abund_CI])

# Propagate the uncertainty in the carbon content and number of individuals for each group
# For cased where no uncertainty projection is available, use the average uncertainty calculate
# above
ciliate_CI = CI_prod_prop(np.array([ciliate_cc_CI, ciliate_abun_CI]))
naked_amoebae_CI = CI_prod_prop(np.array([naked_amoebae_cc_CI,naked_amoebae_abund_CI]))
flagellate_CI = CI_prod_prop(np.array([average_cc_CI,average_abund_CI]))
testate_amoebae_CI = CI_prod_prop(np.array([testate_amoebae_cc_CI,average_abund_CI]))

# Propagate the uncertainties for each group to the total estimate of the biomass of soil protists
method2_inter_CI = CI_sum_prop(estimates= (carbon_content['Carbon content [g C]']*abund_mean['Abundance [# g^-1]']), 
                               mul_CIs=np.array([ciliate_CI,flagellate_CI,naked_amoebae_CI,flagellate_CI,testate_amoebae_CI]))
print('Our best projection for the interstudy uncertainty associated with the estimate of the total biomass of soil protists based on estimates of carbon content and number of individuals is ≈%.0f-fold' % method2_inter_CI)

### 18S-rDNA sequencing
Our estimate of the biomass of soil protists based on 18S rDNA sequencing relies on data from two studies (Tedersoo et al. and Chen et al.). We calculate the 95% confidence interval around the geometric mean of values from the two studies as our best projection of the interstudy uncertainty associated with the estimate of the total biomass of terrestrial protists based on 18S rDNA sequencing.

In [ ]:
# Calculate the 95% confidence interval around the estimate for the fraction of soil protists
rDNA_frac_CI = frac_CI(np.array([chen_mean,tedersoo_frac]))

print('Our best projection for the interstudy uncertainty associated with the estimate of the total biomass of soil protists based on 18S rDNA sequencing is ≈%.0f-fold' % rDNA_frac_CI)

## Inter-method uncertainty
As our best estimate of the total biomass of soil protists we use the geometric mean of the estimates from the five independent methods. As a measure of the uncertainty associated with the geometric mean of estimates from different methods, we calculate the 95% confidence interval around the geometric mean of the estimates.

Because we are less confident in our estimates based on molecular techmiques, we first calculate the geometric mean of the estimates based on the three molecular techniques, and then calculate the 95% confidence interval of the geometric mean of the estimates from the first two methods and the mean of the estimates based on molecular techniques:

In [ ]:
# Calculate the geometric mean of estimates based on molecular techniques
mol_estimate = gmean([method3_estimate,method4_estimate,method5_estimate])

# Calculate the 95% confidence interval around the geometric mean of values from 
# the two estimates based on direct measurements and the mean value from molecular
# techniques
inter_method_CI = geo_CI_calc(np.array([method1_estimate,method2_estimate,mol_estimate]))

print('Our best projection for the inter-method uncertainty associated with the estimate of the total biomass of soil protists is ≈%.0f-fold' % inter_method_CI)

As our best projection for the uncertainty associated with the estimate of the total biomass of soil protists, we use the highest uncertainty out of the collection of uncetainties we generated at the various levels (intra-study, interstudy, and inter-method):

In [ ]:
mul_CI = np.max([np.max(biomass_study_CI),
                 np.max(ac_CI),
                 frac_CI(chen_data['Protists']),
                 frac_CI(meta_trans_data['Fraction of protists']),
                 np.max(biomass_intra_habitat_CI),
                 biomass_inter_habitat_CI,
                 method2_inter_CI,
                 rDNA_frac_CI,
                 inter_method_CI])

print('Our best projection for the uncertainty associated with the estimate of the total biomass of soil protists is ≈%.0f-fold' % mul_CI)
